In [82]:
# Common standard libraries

import datetime
import time
import os

In [83]:
# Common external libraries

import pandas as pd
import numpy as np
import sklearn # scikit-learn
import requests
from bs4 import BeautifulSoup
from time import sleep


In [84]:
# Visualization libraries

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
# Setting plot appearance
# See here for more options: https://matplotlib.org/users/customizing.html

%config InlineBackend.figure_format='retina'
sns.set() # Revert to matplotlib defaults
plt.rcParams['figure.figsize'] = (9, 6)
plt.rcParams['axes.labelpad'] = 10
sns.set_style("darkgrid")
# sns.set_context("poster", font_scale=1.0)

In [86]:
# Ignore warnings (don't display stderr)

import warnings
warnings.filterwarnings('ignore')

# Circular Health Project
## Mortality Rate Analysis in Italy (2015-2020)
## Dataset generator

This notebook downloads and aggregates the mortality rates data file from a subset of municipalities in the ANPR system</br>
and the municipality census information producing a derived dataset that contains population information and a better</br>
feature sets for analysis. This version uses the ISTAT data format with M_XX, F_XX, T_XX features

Source: https://www.istat.it/it/archivio/240401</br>
Permasource: https://sandbox.zenodo.org/record/535730#.XrJ-v6gzaUk</br>

Version: 1.0-20200504

## Step 1: Load the data from Zenodo

In [87]:
# Load mortality data from ISTAT file
print('Loading mortality data file...')
mr_df = pd.read_csv(filepath_or_buffer='https://zenodo.org/record/3824313/files/comuni_giornaliero_20200504.csv?download=1', header=0, encoding='iso-8859-1')
print('Loaded %d records' %len(mr_df))

Loading mortality data file...
Loaded 914621 records


In [88]:
mr_df.head();

In [89]:
# Load municipalities data from ISTAT file, synch colum names with mortality rates data
print('Loading municipalities population data file based on 2011 census ...')
md_df = pd.read_excel(io='https://zenodo.org/record/3824404/files/Elenco-codici-statistici-e-denominazioni-al-01_01_2020.xls?download=1',
                names=['REG','UNI','PROV_STR','COM_STR','COD_PROVCOM_STR','NOME_COMUNE_INT','NOME_COMUNE','LINGUA','COD_RIP_GEO','RIP_GEO',
                        'NOME_REGIONE','NOME_PROVINCIA','TIPO_COMUNE','SIGLA','COD_PROVCOM','COD_PROVCOM_2016','COD_PROVCOM_2009','COD_PROVCOM_2005',
                        'COD_CAT','POPOLAZIONE','NUTS1','NUTS2(3)','NUTS3'])
print('Loaded %d records' %len(md_df))

Loading municipalities population data file based on 2011 census ...
Loaded 7904 records


In [90]:
md_df.head();

In [91]:
# Load municipalities data from ISTAT file, synch colum names with mortality rates data
print('Loading municipalities population data file updates at January 1st, 2019...')
md_df_2019 = pd.read_csv(filepath_or_buffer='https://zenodo.org/record/3824529/files/comuni-popolazione-residente-2019.csv?download=1', header=1, encoding='iso-8859-1')
print('Loaded %d records' %len(md_df_2019))

Loading municipalities population data file updates at January 1st, 2019...
Loaded 808452 records


In [92]:
md_df_2019.head();

In [93]:
# Load municipalities geographical coordinates
print('Loading municipalities geographical coordinates data file...')
geo_df = pd.read_excel(io='https://zenodo.org/record/3824502/files/italy_geo.xlsx?download=1', sheet_name='italy_geo')
print('Loaded %d records' %len(geo_df))

Loading municipalities geographical coordinates data file...
Loaded 7979 records


In [94]:
geo_df.head();

## Step 2: Look at the data, clean, reorganize

In [95]:
# Identify and NaNs
mr_df.isnull().sum();

In [96]:
# Identify and NaNs
md_df.isnull().sum();

In [97]:
print('Removing records with no valid 2020 data...')

# Remove records for which no 2020 data exists
df_2020 = mr_df[mr_df.T_20 != 'n.d.']

# cells with no record for 2020 are of type string, need to convert first
df_2020 = df_2020.astype({'M_20': int, 'F_20': int, 'T_20': int}) 

print('Records with valid 2020 data:', len(df_2020))

Removing records with no valid 2020 data...
Records with valid 2020 data: 666643


In [98]:
# Generate clean municipality population dataset
def updateTotal(x):
    x['Totale'] = x[0]+x[1];
    return x

col_ETA = md_df_2019.columns[2]
cols_pop = ['Totale Maschi', 'Totale Femmine']
mpop = pd.DataFrame(data=md_df_2019[md_df_2019[col_ETA] != 999].groupby('Codice comune')[cols_pop].sum());
mpop = mpop.apply(lambda x: updateTotal(x), axis=1)
mpop;

In [99]:
from tqdm.notebook import tqdm, trange
from tqdm import tnrange, tqdm_notebook

# Expand dataset to a single death per row
#print('Expanding dataset to single record rows...')

# columns to operate on
#cols_M = ['M_15','M_16','M_17','M_18','M_19','M_20']
#cols_F = ['F_15','F_16','F_17','F_18','F_19','F_20']
#cols = ['M_15','M_16','M_17','M_18','M_19','M_20','F_15','F_16','F_17','F_18','F_19','F_20']
#cols_TOTALE = ['T_15','T_16','T_17','T_18','T_19','T_20']

# first copy the single-record rows to a new dataframe
#df_2020 = df_2020_compact[(df_2020_compact['T_15'] + df_2020_compact['T_16'] + df_2020_compact['T_17'] +
#                           df_2020_compact['T_18'] + df_2020_compact['T_19'] + df_2020_compact['T_20']) == 1];

# then expand multi-record rows
#for _, row in tqdm(df_2020_compact.iterrows(), total=(len(df_2020_compact)-len(df_2020))):
#    if row[cols_TOTALE].sum() != 1:
#        # this row represents multiple death records
#        for i, cellin enumerate(row[cols_M]):
#            # take each M_XX
#            for j in range(1,cell+1):
#                # create new records for each death
#                df_2020 = df_2020.append(row, ignore_index=True)
#                new_vals = [0,0,0,0,0,0,0,0,0,0,0,0]
#                new_vals[i] = 1
#                df_2020.loc[len(df_2020)-1,cols] = new_vals
#        for i, cell in enumerate(row[cols_F]):
#            # take each F_XX
#            for j in range(1,cell+1):
#                # create new records for each death
#                df_2020 = df_2020.append(row, ignore_index=True)
#                new_vals = [0,0,0,0,0,0,0,0,0,0,0,0]
#                new_vals[i+6] = 1
#                df_2020.loc[len(df_2020)-1,cols] = new_vals

#print('Expanded dataset from %d to %d records' %(len(df_2020_compact), len(df_2020)))


In [100]:
# Reorganize the data so that features can be correlated
print('Merging population and geographical info...')
tqdm.pandas()

def getGenderYearPopulationGeo(x):
#    x['GENDER'] = 'Unknown';
#    x['YEAR_OF_DEATH'] = 9999;
    try:
        # try to load info at 01-01-2019
        x['POPULATION'] = mpop[mpop.index == x.COD_PROVCOM].Totale.iloc[0]
    except:
        # fallback to census 2011 (probably the municipality doesn't exist anymore)
        x['POPULATION'] = md_df[md_df.COD_PROVCOM == x.COD_PROVCOM].POPOLAZIONE.iloc[0]
    try:
        x['LONGITUDE'] = geo_df[geo_df.istat == x.COD_PROVCOM].lng.iloc[0]
        x['LATITUDE'] = geo_df[geo_df.istat == x.COD_PROVCOM].lat.iloc[0]
    except:
        x['LONGITUDE'] = 0.0
        x['LATITUDE'] = 0.0

#    if x[['M_15','M_16','M_17','M_18','M_19','M_20']].sum() == 1:
#            x['GENDER'] = 'Male'
#    elif x[['F_15','F_16','F_17','F_18','F_19','F_20']].sum() == 1:
#            x['GENDER'] = 'Female'
#    if (x.M_15 == 1 or x.F_15 == 1):
#        x['YEAR_OF_DEATH'] = 2015
#    elif (x.M_16 == 1 or x.F_16 == 1):
#        x['YEAR_OF_DEATH'] = 2016
#    elif (x.M_17 == 1 or x.F_17 == 1):
#        x['YEAR_OF_DEATH'] = 2017
#    elif (x.M_18 == 1 or x.F_18 == 1):
#        x['YEAR_OF_DEATH'] = 2018
#    elif (x.M_19 == 1 or x.F_19 == 1):
#        x['YEAR_OF_DEATH'] = 2019
#    elif (x.M_20 == 1 or x.F_20 == 1):
#        x['YEAR_OF_DEATH'] = 2020
    
    return x

df_2020 = df_2020.progress_apply(getGenderYearPopulationGeo,axis=1);


Merging population and geographical info...


In [101]:
#cols = ['M_15','M_16','M_17','M_18','M_19','M_20',
#            'F_15','F_16','F_17','F_18','F_19','F_20',
#            'T_15','T_16','T_17','T_18','T_19','T_20']
#df_2020 = df_2020.drop(labels=cols, axis=1)

# Temporary: Remove records with Unknown gender
#df_2020 = df_2020[df_2020.GENDER != 'Unknown'];

# Serialize dataset
print('Saving aggregated dataset')

file_name = 'mortalita_giornaliero_comune_20200504.xlsx'
df_2020.to_excel('mortalita_giornaliero_comune_20200504.xlsx')

print('Aggregated dataset saved as %s' %file_name)

Saving aggregated dataset
Aggregated dataset saved as mortalita_giornaliero_comune_20200504.xlsx
